# Download last.fm listening history

  - To see this analysis live, check out my article ["Analyzing Last.fm Listening History"](http://geoffboeing.com/2016/05/analyzing-lastfm-history/)
  - My last.fm page: http://www.last.fm/user/gboeing
  - API documentation: http://www.last.fm/api
  - For anything more complicated, you might use this Python wrapper for the API: https://github.com/pylast/pylast
  
This tool separately downloads your all-time most played tracks, artists, and albums. Then it downloads all of your scrobbles in order of recency. Each of these 4 data sets are saved to separate CSV files. It downloads the "all-time most" data separately because (at least for my data) my massive iTunes history scrobble-upload in 2007 is included in the all-time most played tracks/artists/albums but is excluded from the recent tracks API endpoint. For accurate analysis of my all-time scrobbles, I need to look at those separate all-time lists, or else 4 years of listening history (from iTunes) is ignored in the calculations.

Replace the "from keys import..." line of code with two new lines of code (replace placeholder values with your actual values):
  1. key=YOUR-LASTFM-API-KEY
  2. username=YOUR-LASTFM-USERNAME

In [1]:
import requests, json, time, pandas as pd

key='edfd8819fb5684d05911d64873c9f3d0'
username='shemer77'

In [2]:
# how long to pause between consecutive API requests
pause_duration = 0.2

## First get your all-time most played tracks, artists, and albums

In [8]:
url = 'https://ws.audioscrobbler.com/2.0/?method=user.get{}&user={}&api_key={}&limit={}&extended={}&page={}&format=json'
limit = 200 #api lets you retrieve up to 200 records per call
extended = 1 #api lets you retrieve extended data for each track, 0=no, 1=yes
page = 1 #page of results to start retrieving at

In [9]:
method = 'toptracks'
request_url = url.format(method, username, key, limit, extended, page)
artist_names = []
track_names = []
play_counts = []
response = requests.get(request_url).json()
for item in response[method]['track']:
    artist_names.append(item['artist']['name'])
    track_names.append(item['name'])
    play_counts.append(item['playcount'])

top_tracks = pd.DataFrame()
top_tracks['artist'] = artist_names
top_tracks['track'] = track_names
top_tracks['play_count'] = play_counts
top_tracks.to_csv('data/lastfm_top_tracks.csv', index=None, encoding='utf-8')
top_tracks.head()

,artist,track,play_count
0,blink-182,Aliens Exist,39
1,Skillet,Monster,36
2,blink-182,Easy Target,34
3,Lil' Wayne,No Worries Feat Detail,32
4,Jay-Z,Hate (feat. Kanye West),31


In [10]:
method = 'topartists'
request_url = url.format(method, username, key, limit, extended, page)
artist_names = []
play_counts = []
response = requests.get(request_url).json()
for item in response[method]['artist']:
    artist_names.append(item['name'])
    play_counts.append(item['playcount'])

top_artists = pd.DataFrame()
top_artists['artist'] = artist_names
top_artists['play_count'] = play_counts
top_artists.to_csv('data/lastfm_top_artists.csv', index=None, encoding='utf-8')
top_artists.head()

,artist,play_count
0,blink-182,538
1,Sum 41,481
2,Silverstein,480
3,Rise Against,467
4,Eminem,429


In [11]:
method = 'topalbums'
request_url = url.format(method, username, key, limit, extended, page)
artist_names = []
album_names = []
play_counts = []
response = requests.get(request_url).json()
for item in response[method]['album']:
    artist_names.append(item['artist']['name'])
    album_names.append(item['name'])
    play_counts.append(item['playcount'])

top_albums = pd.DataFrame()
top_albums['artist'] = artist_names
top_albums['album'] = album_names
top_albums['play_count'] = play_counts
top_albums.to_csv('data/lastfm_top_albums.csv', index=None, encoding='utf-8')
top_albums.head()

,artist,album,play_count
0,Silverstein,A Shipwreck in the Sand (Deluxe Edition),125
1,Silverstein,Rescue,119
2,Vinnie Paz,God of The Serengeti,107
3,Silverstein,This Is How the Wind Shifts,106
4,Vinnie Paz,Season of the Assassin,103


## Now get all your scrobbles

Last.fm provides this 'recenttracks' API method to get 'all' scrobbles. However, it seems to be pretty spotty for data from circa 2007. The best way to determine top tracks, artists, albums is with the cells above. However, the code below retrieves time series data of all scrobbles (but with the caveat of spotty data from 2007 and earlier).

Sample URL: https://ws.audioscrobbler.com/2.0/?method=user.getrecenttracks&user=gboeing&api_key={}&limit=1&extended=0&page=1&format=json

In [12]:
def get_scrobbles(method='recenttracks', username=username, key=key, limit=200, extended=0, page=1, pages=0):
    '''
    method: api method
    username/key: api credentials
    limit: api lets you retrieve up to 200 records per call
    extended: api lets you retrieve extended data for each track, 0=no, 1=yes
    page: page of results to start retrieving at
    pages: how many pages of results to retrieve. if 0, get as many as api can return.
    '''
    # initialize url and lists to contain response fields
    url = 'https://ws.audioscrobbler.com/2.0/?method=user.get{}&user={}&api_key={}&limit={}&extended={}&page={}&format=json'
    responses = []
    artist_names = []
    artist_mbids = []
    album_names = []
    album_mbids = []
    track_names = []
    track_mbids = []
    timestamps = []
    
    # make first request, just to get the total number of pages
    request_url = url.format(method, username, key, limit, extended, page)
    response = requests.get(request_url).json()
    total_pages = response[method]['@attr']['totalPages']
    if pages > 0:
        total_pages = min([total_pages, pages])
        
    print '{} total pages to retrieve'.format(total_pages)
    
    # request each page of data one at a time
    for page in range(1, int(total_pages) + 1, 1):
        if page % 10 == 0: print page,
        time.sleep(pause_duration)
        request_url = url.format(method, username, key, limit, extended, page)
        responses.append(requests.get(request_url))
    
    # parse the fields out of each scrobble in each page (aka response) of scrobbles
    for response in responses:
        scrobbles = response.json()
        for scrobble in scrobbles[method]['track']:
            # only retain completed scrobbles (aka, with timestamp and not 'now playing')
            if 'date' in scrobble.keys():
                artist_names.append(scrobble['artist']['#text'])
                artist_mbids.append(scrobble['artist']['mbid'])
                album_names.append(scrobble['album']['#text'])
                album_mbids.append(scrobble['album']['mbid'])
                track_names.append(scrobble['name'])
                track_mbids.append(scrobble['mbid'])
                timestamps.append(scrobble['date']['uts'])
                
    # create and populate a dataframe to contain the data
    df = pd.DataFrame()
    df['artist'] = artist_names
    df['artist_mbid'] = artist_mbids
    df['album'] = album_names
    df['album_mbid'] = album_mbids
    df['track'] = track_names
    df['track_mbid'] = track_mbids
    df['timestamp'] = timestamps
    df['datetime'] = pd.to_datetime(df['timestamp'].astype(int), unit='s')
    
    return df

In [13]:
# get all scrobbled tracks ever, in order of recency
scrobbles = get_scrobbles()

94 total pages to retrieve
10 20 30 40 50 60 70 80 90


In [14]:
# save the dataset
scrobbles.to_csv('data/lastfm_scrobbles.csv', index=None, encoding='utf-8')
print '{:,} total rows'.format(len(scrobbles))
scrobbles.head()

18,631 total rows


,artist,artist_mbid,album,album_mbid,track,track_mbid,timestamp,datetime
0,Bring Me the Horizon,074e3847-f67f-49f9-81f1-8c8cea147e8e,Sempiternal,86f705ee-242f-4e89-896c-f95bb3044189,Can You Feel My Heart,863dd3b3-c525-427c-a0fb-faa75b1dde37,1463001864,2016-05-11 21:24:24
1,Young Thug,800760de-bdf8-43a2-8fe0-44a2401a5515,Barter 6,,Halftime,,1463001614,2016-05-11 21:20:14
2,Dance Gavin Dance,16456fed-c9f2-4adf-b6ea-97b648c474d2,Instant Gratification,,Death Of A Strawberry,,1463000863,2016-05-11 21:07:43
3,Tech N9ne,dde3d9b1-0e44-48bc-b0c9-d739b3570000,Absolute Power,dea8d7d1-1823-4635-a536-8ca487a91e8c,Here Comes Tecca Nina,f8e3754a-ca1f-494f-ac03-9ceb32545726,1463000622,2016-05-11 21:03:42
4,Bankroll Mafia,,Bankroll Mafia,,"Neg 4 Degrees (feat. Shad Da God, Young Thug &...",,1463000300,2016-05-11 20:58:20
